In [1]:
import os
import numpy as np
import torchvision
import torch
import torchvision.transforms as transforms
from torch.optim import AdamW,Adam
from utils.networkHelper import *
from random import randint

from noisePredictModels.Unet.UNet import Unet
from utils.trainNetworkHelper import SimpleDiffusionTrainer
from diffusionModels.simpleDiffusion.simpleDiffusion import DiffusionModel

In [2]:
def byte2list(byte):
    # convert the byte to an integer representation
    if not type(byte)==int:
        byte = ord(byte)
    # now convert to string of 1s and 0s
    byte = bin(byte)[2:].rjust(8, '0')
    # now byte contains a string with 0s and 1s
    byte=[int(x) for x in byte]
    byte.reverse()
    byte=np.array(byte,dtype=np.uint8)
    return byte

In [3]:
with open(r"D:\Workspace\tpo\ai\spinodal\data\bin\_co_03_10_10_0.bin",'rb') as f:
    a=f.read(1)
    byte2list(a)

In [4]:
root=r'D:\Workspace\tpo\ai\spinodal\data\bin'
paths=os.walk(root)
dataset=[]
for path,dirs,files in paths:
    print(len(files))
    for i in range(10):
        idx=randint(0,5200-1)
        with open(os.path.join(path,files[idx]),'rb') as f:
            data=f.read()
        data=[byte2list(byte) for byte in data]
        data=np.array(data).reshape((1,128,128,128))
        dataset.append(data)
        print(i+1)
dataset=np.array(dataset,dtype=np.uint8)
print(dataset.shape)

1
2
3
4
(4, 1, 128, 128, 128)


In [5]:
dataset=torch.tensor(dataset,dtype=torch.float16)
print(dataset.shape)

torch.Size([4, 1, 128, 128, 128])


In [6]:
image_size = 128
channels = 1
batch_size = 2

data_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=0)


device = "cuda" if torch.cuda.is_available() else "cpu"
dim_mults = (1, 2, 4,)

denoise_model = Unet(
    dim=image_size,
    channels=channels,
    dim_mults=dim_mults
)

timesteps = 100
schedule_name = "linear_beta_schedule"
DDPM = DiffusionModel(schedule_name=schedule_name,
                      timesteps=timesteps,
                      beta_start=0.0001,
                      beta_end=0.02,
                      denoise_model=denoise_model).to(device)

optimizer = AdamW(DDPM.parameters(), lr=1e-3)
epoches = 20

Trainer = SimpleDiffusionTrainer(epoches=epoches,
                                 train_loader=data_loader,
                                 optimizer=optimizer,
                                 device=device,
                                 timesteps=timesteps)

# 训练参数设置
root_path = "./saved_train_models"
setting = "imageSize{}_channels{}_dimMults{}_timeSteps{}_scheduleName{}".format(image_size, channels, dim_mults, timesteps, schedule_name)

saved_path = os.path.join(root_path, setting)
if not os.path.exists(saved_path):
    os.makedirs(saved_path)

DDPM = Trainer(DDPM, model_save_path=saved_path)

  0%|          | 0/4 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1024.00 MiB. GPU 0 has a total capacty of 8.00 GiB of which 0 bytes is free. Of the allocated memory 21.43 GiB is allocated by PyTorch, and 18.06 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF